In [26]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

import os
import sys
import copy 
from time import time

from astropy.table import Table, vstack

import numpy as np 
import emcee

import scipy.ndimage
from scipy.ndimage.filters import gaussian_filter

import matplotlib.pyplot as plt
import matplotlib.mlab as ml
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

from astroML.stats import binned_statistic_2d

from cap_loess_2d import loess_2d
from cap_plot_velfield import plot_velfield

plt.rcParams['figure.dpi'] = 100.0
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12.0
plt.rc('text', usetex=True)
    
from asap.um_model_plot import plot_mtot_minn_smf, plot_dsigma_profiles
from asap.asap_data_io import parse_config, load_observed_data, \
    config_observed_data, config_um_data, load_um_data
from asap.asap_utils import mcmc_save_chains, mcmc_save_results, \
    mcmc_initial_guess, mcmc_samples_stats, mcmc_save_results
from asap.asap_model_setup import setup_model
from asap.asap_likelihood import asap_flat_prior, asap_ln_like, asap_flat_prior_transform

from asap.asap_run_model import * 
from asap.asap_model_prediction import *
from asap.asap_likelihood import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from numpy import linalg

import dynesty
from dynesty import plotting as dyplot

### Read the configuration and data

In [35]:
config_file = '../asap/asap_default_config.yaml'

cfg_ini = parse_config(config_file)

global cfg, obs_data, um_data

cfg, obs_data, um_data = initial_model(cfg_ini, verbose=True)

# Stellar mass catalog: s16a_wide2_massive_fsps1_imgsub_use_short.fits
# Weak lensing profile sample: s16a_wide2_massive_boxbin3_default
# Pre-computed SMF for inner logMs: ../data/s16a_massive_wide2/smf/s16a_wide2_massive_smf_m10_11.5.fits
# Pre-computed SMF for total logMs: ../data/s16a_massive_wide2/smf/s16a_wide2_massive_smf_mmax_11.5.fits
# Using logm_10 as inner stellar mass.
# Using logm_max as total stellar mass.
# There are 15 weak lensing profiles in this sample
# SMF for total stellar mass: 
  11.5000 -- 12.4000 in 9 bins
# SMF for inner stellar mass: 
  10.8000 -- 11.8000 in 10 bins
# Pre-computed full SMF: primus_smf_z0.3_0.4.fits
# For inner stellar mass: 
    10 bins at 10.80 < logMinn < 11.80
# For total stellar mass: 
    9 bins at 11.50 < logMtot < 12.40
# The volume of the HSC data is    102060083.62 Mpc^3
# The volume of the UniverseMachine mock is    205348196.23 Mpc^3


In [40]:
print(cfg.keys())

['um_min_scatter', 'param_ini', 'mcmc_out_dir', 'obs_smf_n_data', 'obs_wl_sample', 'obs_omega_m', 'obs_smf_tot', 'obs_smf_full_fits', 'smf_tot_file', 'um_mtot_nbin_min', 'param_sig', 'mcmc_wl_only', 'mcmc_nsamples', 'um_model', 'obs_smf_inn', 'obs_h0', 'obs_smf_full_file', 'obs_minn_col', 'um_wl_add_stellar', 'obs_dir', 'obs_area', 'um_redshift', 'obs_smf_inn_nbin', 'mcmc_wl_weight', 'um_wl_nbin', 'obs_cosmo', 'obs_volume', 'um_cosmo', 'obs_smf_inn_min', 'obs_cat', 'obs_smf_tot_nbin', 'mcmc_run_file', 'mcmc_prefix', 'mcmc_smf_only', 'mcmc_nthreads', 'obs_wl_n_bin', 'um_dir', 'obs_smf_tot_min', 'smf_inn_file', 'um_min_mvir', 'mcmc_labels', 'um_lbox', 'um_mtot_nbin', 'um_wl_cat', 'param_upp', 'obs_z_col', 'um_omega_m', 'param_low', 'um_min_nobj_per_bin', 'obs_dsigma_n_data', 'um_star_col', 'um_wl_minr', 'obs_mtot_col', 'um_halo_col', 'obs_smf_inn_max', 'mcmc_ndims', 'um_h0', 'mcmc_nwalkers', 'um_volume', 'um_wl_maxr', 'mcmc_burnin_file', 'model_type', 'model_dof', 'obs_wl_out', 'mcmc_nbu

### Test flat prior transform

In [14]:
from prospect.models import priors

a = priors.TopHat(mini=-0.1, maxi=3.0)

In [41]:
unit_cube = np.random.random(cfg_ini['mcmc_ndims'])

asap_flat_prior_transform(unit_cube, cfg['param_low'], cfg['param_upp'])

array([ 0.58133591,  6.6947321 , -0.12088829,  0.06932583,  0.47029372,
        0.12406602])

### Test dynesty sampling

In [46]:
sampler = dynesty.NestedSampler(asap_ln_like, 
                                asap_flat_prior_transform, 
                                cfg['mcmc_ndims'],
                                logl_args=[cfg, obs_data, um_data],
                                ptform_args=[cfg['param_low'], cfg['param_upp']],
                                bound='single', 
                                nlive=100)

In [ ]:
sampler.run_nested(dlogz=0.01)
res = sampler.results

iter: 1253 | bound: 111 | nc: 25 | ncall: 13266 | eff(%):  9.445 | loglstar:   -inf < -240.012 <    inf | logz: -253.953 +/-  0.357 | dlogz: 12.985 >  0.010               

In [ ]:
# Plot results.
lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res, lnz_truth=lnz_truth)